# LKIPA Resonance Measurement

In this notebook I develop the code for running a resonance measurement on the LKIPA for different DC Bias settings.

- **DC Bias**
    - Value: For each run, single value $\in \{0.0, ..., 5.0\} \text{V}$
    
- **Signal** 
    - Amplitude: Single value, variable (e.g., 0.04 FS)
    - Frequency: For each run, single value $\in \{4.35, ..., 4.45\}$ GHz, $\Delta \omega$ variable

- **Pump**
    - Amplitude: OFF
    - Frequency: OFF

- **Output**
    - Frequency: Same as Signal






In [93]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys
import math
import presto
from presto import lockin, utils, hardware
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode

## 1. Format for saving measurement data

In [94]:
# Save attributes function
import string


def save_attr(folder, file, sample_, myrun_, myrun_attrs_):
    if not os.path.isdir(folder):
        os.makedirs(folder)
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        run_str = f"{myrun_}"
        # Assicurati che il gruppo esista
        if run_str not in savefile:
            savefile.create_group(run_str)
        for key, val in myrun_attrs_.items():
            savefile[run_str].attrs[key] = val
    print("Saved run attributes.")

dt = h5py.string_dtype(encoding='utf-8')

# Save data function
def save_data(folder, file, sample, myrun, freq, df, usb_arr, DC_Bias):
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # Open the save file (.hdf5) in append mode
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        # String as handles
        freq_data_str = "{}/freq comb".format(myrun)
        df_data_str = "{}/df".format(myrun)
        usb_data_str = "{}/USB".format(myrun)
        sample_data_str = "{}/Sample".format(myrun)
        DCB_data_str = "{}/DC Bias".format(myrun)

        # Write data to datasets
        savefile.create_dataset(
            freq_data_str, 
            (np.shape(freq)),
            dtype=float, 
            data=freq
            )
        
        savefile.create_dataset(
            df_data_str, 
            (np.shape(df)),
            dtype=float, 
            data=df
            )
        
        savefile.create_dataset(
            usb_data_str, 
            (np.shape(usb_arr)),
            dtype=complex, 
            data=usb_arr
            )
        
        savefile.create_dataset(
            sample_data_str, 
            None,
            dtype=dt, 
            data=sample
            )
        
        savefile.create_dataset(
            DCB_data_str, 
            (np.shape(DC_Bias)),
            dtype=float,
            data=DC_Bias)

        # Write dataset attributes
        savefile[freq_data_str].attrs["Unit"] = "Hz"
        savefile[df_data_str].attrs["Unit"] = "Hz"
        savefile[usb_data_str].attrs["Unit"] = "fsu complex"
        savefile[DCB_data_str].attrs["Unit"] = "V"

## 2. Setting up the measurement parameters for Presto

In [95]:
### MEASUREMENT SETUP ###

# Filename based on timestamp of experimental run
Ym_str = time.strftime("%Y-%m")                                 # Get year and month
meas_type   = 'Scattering'                                      # Measurement type
save_folder = r'I:/LKiPA-Data/{}/{}'.format(Ym_str, meas_type)  # Create folder for current month and measurement type
myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")                # Save experimental run for each timestamp
save_file   = r"{}.hdf5".format(myrun)                          # Save data in hdf5 file for current run

sample      = 'LKIPA'
atten       = 80
temperature = 0.0096 # Take latest value


# Network settings for Presto Hardware
ADDRESS = '130.237.35.90'   # IP Address
PORT    = 42873             # TCP Port
Box     = 'Presto DELTA'    # Model Identifier

# Port Assignments
input_port   = 5 # Output from LKIPA (to be measured)
output_port  = 8 # Signal to LKIPA (vacuum for correlation experiments)
flux_port    = 2 # Pump frequency comb 
bias_port    = 2 # DC Bias for optimal operating point of JPA (based on dc bias-resonant frequency curve)

# Pseudorandom noise (only when working with small amplitudes)
dither = False

# DAC current
DAC_CURRENT  = 32_000  # µA : Max current allowed for DC Bias 

# DC bias values
bias_vals     = np.arange(0, 5, 0.5)   # FS  From calibration   4.50 V --> f0= 4424855567.651496 Hz  

In [96]:
### MEASUREMENT PARAMETERS ###

# NCO frequencies (Numerically Controlled Oscillator) : Digital Local Oscillator
fNCO = 4.21e9        # NCO frequency for output comb and signal
_df_sweep = 150e3  # Default = 100 kHz

# Number of pixels
_Npix = 5_000
Navg = 1 # No averaging
Nskip = 0 # Number of pixels we discard


## SIGNAL PARAMETERS ##
# Signal output amplitude list from Presto [FS] (Never exceed 0.05FS for OUTPORT!!
sig_amp = 0.7

#sig_amp_step = sig_amp_min
sig_amp_arr = np.array([sig_amp]) #np.arange(sig_amp_min, sig_amp, sig_amp_step)
sig_amp_num = len(sig_amp_arr) # = 1 for no signal amplitude sweep

# Signal Frequency list
sig_freq_num = 95 # Number of frequencies of the frequency comb
sig_freq_step = _df_sweep  # 100 kHz
sig_freq_centre = 4.406e9  # center
sig_freq_arr = sig_freq_centre + sig_freq_step*np.arange(
    -(sig_freq_num+1)/2+1,
    (sig_freq_num+1)/2, 
    1
    )

sig_freq_min = sig_freq_arr[0]
sig_freq_max = sig_freq_arr[-1]             

# Propagation Phase Drift
input_phases = np.zeros(sig_freq_num)
dPhiDrift_df = 1.8869636024e-6  # rad/Hz - Measured Drift in phase per Hz

## 3. Instantiate ```lockin``` mode and **RUN** experiment

In [97]:
# Instantiate lockin device
with lockin.Lockin(
    address=ADDRESS,
    port=PORT,
    adc_mode=AdcMode.Mixed,
    adc_fsample=AdcFSample.G2,
    dac_mode=[DacMode.Mixed04, DacMode.Mixed02, DacMode.Mixed02, DacMode.Mixed02],
    dac_fsample=[DacFSample.G10, DacFSample.G6, DacFSample.G6, DacFSample.G6],
    ) as lck:

    # Start timer
    t_start = time.strftime("%Y-%m-%d_%H_%M_%S")

    # Print Presto version
    print("Presto version: " + presto.__version__)

    # DAC current
    lck.hardware.set_dac_current(output_port, DAC_CURRENT)

    # Initialise data array
    usb_arr = np.zeros((len(bias_vals), sig_freq_num), dtype=complex)

    # Bandwidth array and frequency combs arrays (to save): Although the arrays can be hardcoded as 1x1 arrays, this format allows for code reuse.
    df = 0
    fs_comb_arr = np.zeros((sig_freq_num))
    fp_center = 0

    # Configure mixer just to be able to create output and input groups
    lck.hardware.configure_mixer(
        freq=fNCO,
        in_ports=input_port,
        out_ports=output_port,
        sync=True,
        )
    
    # Create output group for the signal
    og = lck.add_output_group(ports=output_port, nr_freq=sig_freq_num)
    og.set_amplitudes(sig_amp_arr)
    og.set_phases(
        phases=0.0,
        phases_q=-np.pi / 2
        )
    
    # Create input group for measurement
    ig = lck.add_input_group(port=input_port, nr_freq=sig_freq_num)

    # Apply settings and let system relax
    lck.apply_settings()
    lck.hardware.sleep(1e-4, False)

    ### RUN EXPERIMENT ###
    with tqdm(total=(sig_freq_num * len(bias_vals)), ncols=80) as pbar:        
        # Set measurement comb
        _fs_center = sig_freq_centre - fNCO

        # Tune center frequency
        fs_center, df = lck.tune(_fs_center, _df_sweep)
        
        # Signal comb
        fs_comb = fs_center + df * np.arange(
            -(sig_freq_num+1)/2+1,
            (sig_freq_num+1)/2
            )

        # Set df
        lck.set_df(df)

        # Set input phases
        input_phases[0]=0.0
        for i in range(1, sig_freq_num):
            input_phases[i] = input_phases[i-1] - dPhiDrift_df * df  # the minus work
        input_phases_q =  input_phases - np.pi /2 
        ig.set_phases(phases=input_phases.copy()) 

        # Sweep each DC Bias value
        for dcb_idx, dcb_val in enumerate(bias_vals):
            lck.hardware.set_dc_bias(dcb_val, bias_port)
            lck.hardware.sleep(1.0, False) # Let the system reach steady state

            # Sweep each signal frequency
            for freq_idx, freq_val in enumerate(fs_comb):
                # Set Signal (probe) Frequency
                og.set_frequencies(freq_val) # signal
                ig.set_frequencies(freq_val)
                lck.apply_settings()    

                # get scattering data
                Npix = math.floor(_Npix)
                data = lck.get_pixels(Npix + Nskip, summed=False, nsum=Navg)
                freqs, pixels_i, pixels_q = data[input_port]

                # Convert a measured IQ pair into a low/high si
                # deband pair
                LSB, HSB = utils.untwist_downconversion(pixels_i, pixels_q)
                usb_arr[dcb_idx, freq_idx] = np.mean(HSB)

                # Update progress bar
                pbar.update(1)

        # Mute outputs at the end of the sweep
        og.set_amplitudes(0.0)
        lck.apply_settings()
        lck.hardware.set_dc_bias(0.0, bias_port)

    # Stop timer
    t_end = time.strftime("%Y-%m-%d_%H_%M_%S")

    # Create dictionary with attributes
    myrun_attrs = {
        "Meas": meas_type,
        "Instr": Box,
        "T": temperature,
        "Sample": sample,
        "att": atten,
        "4K-amp_out": 42,
        "RT-amp_out": 41,
        "RT-amp_in": 0,
        "nr_sig_freqs": sig_freq_num,
        "amp_sig": sig_amp_arr,
        "DC bias": bias_vals,
        "Npixels": Npix,
        "Naverages": Navg,
        "Nskip": Nskip,
        "Dither": dither,
        "t_start": t_start,
        "t_meas": t_end
        }
    
    # Save data
    save_data(
        save_folder, 
        save_file, 
        sample, 
        myrun, 
        fs_comb + fNCO, 
        df, 
        usb_arr,
        bias_vals
        )
    
    print('Data Saved')

Presto version: 2.16.0


  0%|                                                   | 0/950 [00:00<?, ?it/s]

100%|█████████████████████████████████████████| 950/950 [06:13<00:00,  2.55it/s]

Data Saved
